In [ ]:
import warnings
import os
import sys

warnings.filterwarnings('ignore')
current_dir = %pwd

parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))

sys.path.append(parent_dir)

In [ ]:
from src.main.models.logistic import Logistic
from src.main.utilities.utils import clean_text, stop_words_removal, stemming, tfidf_vectorizer, get_dataset
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# define model and pipeline
logistic = Logistic(verbose=False, multi_class="multinomial", n_jobs=-1, max_iter=1000)
logistic.set_pipeline([clean_text, stop_words_removal, stemming, tfidf_vectorizer])

# import dataset and apply pipeline
inputs, targets = get_dataset()
inputs = logistic.run_pipeline(inputs, save=True)

In [ ]:

# stratified train test split
x_train, x_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.2, random_state=42, stratify=targets)

# print(y_train.value_counts(normalize=True) * 100)

In [ ]:
# model training 
# search = logistic.grid_search(x_train, y_train, n_iter=5)
# search.best_estimator_
logistic.fit(x_train, y_train)

In [ ]:
# search.cv_results_
# search.best_score_

In [ ]:
# model predictions

y_pred = logistic.predict(x_test)

results = logistic.compute_performance(y_test, y_pred)
logistic.print_performance(results)


In [ ]:
logistic.plot_confusion_matrix(y_test, y_pred)

In [ ]:
logistic.plot_performance(results)

In [ ]:
from sklearn.metrics import get_scorer_names
get_scorer_names()

In [ ]:
import multiprocessing

multiprocessing.cpu_count()

In [ ]:
import socket

hostname = socket.gethostname()
ip_address = socket.gethostbyname(hostname)

print("Hostname:", hostname)
print("IP Address:", ip_address)